In [ ]:
%cd ..

In [ ]:
!pip install uv

In [ ]:
!uv pip install -r requirements.txt --system

In [ ]:
from typing import Literal
from model_downloads import download_qwen_models, download_zimage_models, download_wan22_models

SELECTED: Literal["qwen-image-edit", "qwen-image", "zimage-turbo", "wan22-i2v", "wan22-t2v"] = "qwen-image-edit"

qwen_image_edit_path, qwen_image_path, zimage_turbo_path, wan22_i2v_path, wan22_t2v_path = None, None, None, None, None

if SELECTED.startswith("qwen"):
  qwen_image_edit_path, qwen_image_path = download_qwen_models()
elif SELECTED.startswith("zimage"):
  zimage_turbo_path = download_zimage_models()
elif SELECTED.startswith("wan22"):
  wan22_i2v_path, wan22_t2v_path = download_wan22_models()

In [ ]:
import ctypes

try:
    ctypes.CDLL("libcuda.so.1", mode=ctypes.RTLD_GLOBAL)
    print("✅ Successfully loaded real CUDA driver (libcuda.so.1)")
except OSError:
    print("⚠️ WARNING: Could not load libcuda.so.1 manually.")

In [ ]:
from models.lightx2v_models import QwenImageLite, QwenImageEditLite, ZImageTurbo, Wan22Lite

if SELECTED == "qwen-image-edit":
  gen_type = "i2i"
  model = QwenImageEditLite(model_path=str(qwen_image_edit_path))
elif SELECTED == "qwen-image":
  gen_type = "t2i"
  model = QwenImageLite(model_path=str(qwen_image_path))
elif SELECTED == "zimage-turbo":
  gen_type = "t2i"
  model = ZImageTurbo(model_path=str(zimage_turbo_path))
else:
  gen_type = "t2v" if SELECTED == "wan22-t2v" else "i2v"
  model = Wan22Lite(
    model_path=str(wan22_i2v_path if gen_type == "i2v" else wan22_t2v_path), 
    generation_type=gen_type, 
    quant_scheme="fp8-sgl",
    text_encoder_quantized=True,
  )

In [ ]:
from pathlib import Path
import requests

def download_image(image_url: str, output_path: str, override=False):
  if Path(output_path).exists() and not override:
    return

  response = requests.get(image_url)
  response.raise_for_status()
  with open(output_path, "wb") as f:
    f.write(response.content)


In [ ]:
import time

from IPython.display import Video, display
from core.utils import image_tensor_to_pil, save_video_tensor

input_image_path_1 = "img_in_1"
output ="img_out.png"

is_video = gen_type.endswith("v")
    
support_input_img = gen_type in ["i2i", "i2v", "i2v"]
if support_input_img:
  download_image(
    "https://plus.unsplash.com/premium_photo-1690407617542-2f210cf20d7e?fm=jpg&q=60&w=3000&ixlib=rb-4.1.0&ixid=M3wxMjA3fDB8MHxzZWFyY2h8MXx8cGVyc29ufGVufDB8fDB8fHww",
    input_image_path_1,
  )

t = time.perf_counter()
output = model.generate(
  prompt="Make this person look like a robot",
  image_paths=[input_image_path_1] if support_input_img else [],
  aspect_ratio="1:1",
  resolution="1K",
  seed=9999,
)
print(f"Generated in {time.perf_counter() - t:.2f}s")

assert output is not None, "Oops, something went wrong"

if is_video:
    output_path = "video.mp4"
    save_video_tensor(output, output_path, fps=16)
    display(Video(output, embed=True))
else:
    display(image_tensor_to_pil(output))

---
### Media Scoring
---

In [ ]:
from typing import Literal
import time
from core.services.media_scoring import ImageScorer, VideoScorer
from model_downloads import download_image_scorer, download_video_scorer

scoring_typ: Literal["video", "image"] = "video"

t = time.perf_counter()
if scoring_typ == "video":
  scorer = VideoScorer(download_video_scorer())
else:
  scorer = ImageScorer(download_image_scorer())
print(f"Scorer loaded in {time.perf_counter() - t:.2f}s")

In [ ]:
import time

t = time.perf_counter()
scores = scorer.score(["videotest.mp4"])
print(f"Scores: {scores}")
print(f"Scored in {time.perf_counter() - t:.2f}s")

#### Score grid

In [ ]:
from pathlib import Path
import base64
import mimetypes

from IPython.display import HTML, display
import gc
import torch

def free_mem():
    for _ in range(3):
        torch.cuda.empty_cache()
        gc.collect()
        time.sleep(0.2)

if scoring_typ == "video":
    video_dir = Path(".")
    video_paths = sorted(
        list(video_dir.glob("*.mp4")) +
        list(video_dir.glob("*.webm"))
    )

    items_html = {}
    torch.cuda.reset_peak_memory_stats()

    for path in video_paths:
        free_mem()
        score = scorer.score([str(path)], fps=16)[0]
        free_mem()

        mime_type, _ = mimetypes.guess_type(path)
        if mime_type is None:
            continue

        video_bytes = path.read_bytes()
        video_b64 = base64.b64encode(video_bytes).decode("utf-8")

        items_html[score] = f"""
        <div style="display:flex; flex-direction:column; align-items:center;">
            <video
                controls
                style="max-height:400px; width:100%; object-fit:contain;"
            >
                <source src="data:{mime_type};base64,{video_b64}" type="{mime_type}">
            </video>
            <div style="margin-top:6px; font-weight:600;">
                Score: {score:.4f}
            </div>
        </div>
        """

    peak_alloc = torch.cuda.max_memory_allocated()
    peak_reserved = torch.cuda.max_memory_reserved()

    print(f"Peak allocated: {peak_alloc / 1024**2:.1f} MB")
    print(f"Peak reserved:  {peak_reserved / 1024**2:.1f} MB")
    
    items_html = dict(
        sorted(items_html.items(), key=lambda item: item[0], reverse=True)
     )

    grid_html = f"""
    <div style="
        display:grid;
        grid-template-columns: repeat(3, 1fr);
        gap:16px;
    ">
        {''.join(items_html.values())}
    </div>
    """

    display(HTML(grid_html))
else:
    image_dir = Path(".")
    image_paths = sorted(
        list(image_dir.glob("*.jpg")) +
        list(image_dir.glob("*.jpeg")) +
        list(image_dir.glob("*.png")) +
        list(image_dir.glob("*.webp"))
    )

    items_html = {}
    torch.cuda.reset_peak_memory_stats()

    for path in image_paths:
        free_mem()
        score = scorer.score([str(path)])[0]
        free_mem()

        mime_type, _ = mimetypes.guess_type(path)
        if mime_type is None:
            continue

        image_bytes = path.read_bytes()
        image_b64 = base64.b64encode(image_bytes).decode("utf-8")

        items_html[score] = f"""
        <div style="display:flex; flex-direction:column; align-items:center;">
            <img
                src="data:{mime_type};base64,{image_b64}"
                style="max-height:400px; width:100%; object-fit:contain;"
            />
            <div style="margin-top:6px; font-weight:600;">
                Score: {score:.4f}
            </div>
        </div>
        """

    peak_alloc = torch.cuda.max_memory_allocated()
    peak_reserved = torch.cuda.max_memory_reserved()

    print(f"Peak allocated: {peak_alloc / 1024**2:.1f} MB")
    print(f"Peak reserved:  {peak_reserved / 1024**2:.1f} MB")
    
    items_html = dict(
        sorted(items_html.items(), key=lambda item: item[0], reverse=True)
     )
    
    grid_html = f"""
    <div style="
        display:grid;
        grid-template-columns: repeat(3, 1fr);
        gap:16px;
    ">
        {''.join(items_html.values())}
    </div>
    """

    display(HTML(grid_html))